#   Insights STARS WARS - Teste Prático Globo

In [9]:
import os
import shutil
import duckdb

duckdb_conn = duckdb.connect(database=":memory:", read_only=False)

def mover_arquivos_parquet():
    if not os.path.exists('parquet'):
        os.makedirs('parquet')
    [shutil.move(f, 'parquet') for f in os.listdir('.') if f.endswith('.parquet')]

mover_arquivos_parquet()

df_films = duckdb_conn.from_parquet("./parquet/films_data.parquet").to_df()
df_people = duckdb_conn.from_parquet("./parquet/people_data.parquet").to_df()
df_films = duckdb_conn.from_parquet("./parquet/films_data.parquet").to_df()
df_starships = duckdb_conn.from_parquet("./parquet/starships_data.parquet").to_df()
df_vehicles = duckdb_conn.from_parquet("./parquet/vehicles_data.parquet").to_df()
df_planets = duckdb_conn.from_parquet("./parquet/planets_data.parquet").to_df()
df_species = duckdb_conn.from_parquet("./parquet/species_data.parquet").to_df()

In [22]:
query = """
SELECT *
FROM df_species as s
limit 5
;

"""
result = duckdb_conn.execute(query)
result.df()

,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld,language,people,films,created,edited,url,detail
0,Human,mammal,sentient,180,"caucasian, black, asian, hispanic","blonde, brown, black, red","brown, blue, green, hazel, grey, amber",120,https://swapi.dev/api/planets/9/,Galactic Basic,"[https://swapi.dev/api/people/66/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T13:52:11.567000Z,2014-12-20T21:36:42.136000Z,https://swapi.dev/api/species/1/,None
1,Droid,artificial,sentient,n/a,n/a,n/a,n/a,indefinite,None,n/a,"[https://swapi.dev/api/people/2/, https://swap...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:16:16.259000Z,2014-12-20T21:36:42.139000Z,https://swapi.dev/api/species/2/,None
2,Wookie,mammal,sentient,210,gray,"black, brown","blue, green, yellow, brown, golden, red",400,https://swapi.dev/api/planets/14/,Shyriiwook,"[https://swapi.dev/api/people/13/, https://swa...","[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T16:44:31.486000Z,2014-12-20T21:36:42.142000Z,https://swapi.dev/api/species/3/,None
3,Rodian,sentient,reptilian,170,"green, blue",n/a,black,unknown,https://swapi.dev/api/planets/23/,Galatic Basic,[https://swapi.dev/api/people/15/],[https://swapi.dev/api/films/1/],2014-12-10T17:05:26.471000Z,2014-12-20T21:36:42.144000Z,https://swapi.dev/api/species/4/,None
4,Hutt,gastropod,sentient,300,"green, brown, tan",n/a,"yellow, red",1000,https://swapi.dev/api/planets/24/,Huttese,[https://swapi.dev/api/people/16/],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T17:12:50.410000Z,2014-12-20T21:36:42.146000Z,https://swapi.dev/api/species/5/,None


## Distribuição de Espécies por Planeta

In [20]:
query = """
SELECT p.name as Planeta, s.name as Especie, COUNT(s.name) AS Qtd_Especie
FROM df_species as s
JOIN df_planets as p
ON s.homeworld = p.url
GROUP BY p.name, s.name;

"""
result = duckdb_conn.execute(query)
result.df()

,Planeta,Especie,Qtd_Especie
0,Tund,Toong,1
1,Iktotch,Iktotchi,1
2,Skako,Skakoan,1
3,Muunilinst,Muun,1
4,Endor,Ewok,1
5,Coruscant,Human,1
6,Nal Hutta,Hutt,1
7,unknown,Yoda's species,1
8,Sullust,Sullustan,1
9,Troiken,Xexto,1


## Relação entre Espécies e Filmes

In [37]:

query = """
SELECT
    s.name AS Especie,
    COUNT(DISTINCT f.title) AS Qtd_Filmes
FROM
    df_species as s
JOIN
    df_films as f 
ON ARRAY_CONTAINS(f.species, s.url)
GROUP BY
    s.name

"""
result = duckdb_conn.execute(query)
result.df()

,Especie,Qtd_Filmes
0,Muun,2
1,Geonosian,2
2,Kaleesh,1
3,Hutt,2
4,Xexto,1
5,Tholothian,2
6,Iktotchi,2
7,Human,6
8,Gungan,2
9,Mirialan,2


## Comparação de Características Físicas

In [56]:
query = """
SELECT
    classification as Classificação,
    ROUND(AVG(TRY_CAST(REPLACE(REPLACE(REPLACE(average_height, 'unknown', '0'), 'n/a', '0'), 'indefinite', '0') AS FLOAT)),2) AS Altura_Media,
    ROUND(AVG(TRY_CAST(REPLACE(REPLACE(REPLACE(average_lifespan, 'unknown', '0'), 'n/a', '0'), 'indefinite', '0') AS FLOAT)),2) AS Expectativa_Med_Vida
FROM
    df_species
WHERE
    classification not in ('unknown','n/a','indefinite') 
GROUP BY
    classification;
"""
result = duckdb_conn.execute(query)
result.df()


,Classificação,Altura_Media,Expectativa_Med_Vida
0,sentient,170.00,0.00
1,reptile,150.00,53.00
2,amphibian,186.33,37.50
3,insectoid,178.00,0.00
4,mammal,144.75,155.69
5,artificial,0.00,0.00
6,reptilian,180.00,70.00
7,gastropod,300.00,1000.00
8,mammals,200.00,0.00


## Idiomas

In [53]:
query = """
SELECT
    language as Lingua,
    COUNT(DISTINCT name) AS Qtd_Especies
FROM
    df_species
WHERE language not in('unknown', 'n/a') 
GROUP BY
    language;

"""
result = duckdb_conn.execute(query)
result.df()

,Lingua,Qtd_Especies
0,vulpterish,1
1,Twi'leki,1
2,besalisk,1
3,Gungan basic,1
4,Dugese,1
5,Xextese,1
6,Dosh,1
7,Toydarian,1
8,Quermian,1
9,Sullutese,1


## Qtde de Especies por Filmes

In [60]:
query = """
SELECT
    f.title AS Filme,
    COUNT(DISTINCT s.name) AS Qtde_Especie
FROM
    df_species as s
JOIN
    df_films as f 
ON ARRAY_CONTAINS(f.species, s.url)
GROUP BY
    f.title;

"""
result = duckdb_conn.execute(query)
result.df()

,Filme,Qtde_Especie
0,The Phantom Menace,20
1,A New Hope,5
2,Return of the Jedi,9
3,Attack of the Clones,14
4,The Empire Strikes Back,5
5,Revenge of the Sith,20
